In [ ]:
import os
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
from scipy import optimize

In [ ]:
data_dir = '/Akamai/voice/data/ground-estimate/'
data2_dir = '/Akamai/voice/data/pitches-vuv-crepe-assisted/'
data3_dir = '/Akamai/voice/data/pitches-postprocessed/crepe/'
data4_dir = '/Akamai/voice/data/pitch-overrides/crepe/'

In [ ]:
def shift_chunk(y, ishift, graph=False, gurian=False):
    """
    Shifts an array to the right by *ishift* indexes.
    Fills with float 0's.
    
    shift: an integer number of indexes to shift right
    """
    
    if ishift == 0:
        return y
    
    if ishift > 0:
        return np.r_[np.zeros(ishift), y[:-ishift]]
    else:
        print(ishift, len(y[-ishift:]), np.zeros(-ishift))
        return np.r_[y[-ishift:], np.zeros(-ishift)]
        
def shift_array(y, shifts, graph=False, gurian=False):
    """Shifts different chunks of an array by different amounts.
       Fills each chunk with float 0's.
       
       shifts: tuples (startx, endx, shift) where
        - startx is a float, starting time in seconds
        - endx is a float, ending time in seconds (exclusive)
        - shift is a float, shift amount (to the right) in seconds
    """
    chunks = np.empty(0)
    for start, end, shift in shifts:
        y1 = shift_chunk(y[round(100*start):round(100*end)], round(100*shift))
        chunks = np.append(chunks, y1)
    return chunks

In [ ]:
def parse_name(song, vtype):
    ans = None
    if "AHDS1M" in vtype:
        ans = 0
    elif "AHDS2M" in vtype:
        ans = 1
    elif "AHDS3M" in vtype:
        ans = 2
    
    if "GVM009" in song and ans is not None:
        return 2 - ans
    return ans

In [ ]:
def shift_songs(data_dir, shift_dir, collection, findtext, match=None):
    for song in os.listdir(os.path.join(data_dir, collection)):        
        if song != working_song and working_song != "": continue
        if "checkpoints" in song: continue
        print('\t\t', song)
        shiftsong = song
        if collection == "Scherbaum Mshavanadze":
            shiftsong = song[:6]
        with open(os.path.join(shift_dir, collection, shiftsong, "shifts.txt")) as shift_file:
            for voice_type in os.listdir(os.path.join(data_dir, collection, song)):
                if "shifts" in voice_type or "shifted" in voice_type or "~" in voice_type or "#" in voice_type: continue
                if parse_name(song, voice_type) is None: continue
                if match != None and (not match in voice_type): continue
                print(voice_type)
                    
                shiftList = list(map(float, shift_file.readline().split(' ')))
                it = iter(shiftList)
                shifts = list(zip(it, it, it))
                
                cdata = np.loadtxt(os.path.join(data_dir, collection, song, voice_type))
                x1, y1 = cdata[:, 0], cdata[:, 1]
                y2 = shift_array(y1, shifts)
                outfn = os.path.join(data_dir, collection, song, voice_type[:voice_type.index(findtext)] + "_shifted.txt")
                np.savetxt(outfn, np.vstack((x1, y2)).T, fmt='%.2f')
    

In [ ]:
def chunkIndex(c, shifts):
    chunk = 0
    for start, end, shift in shifts:
        if start <= c[0] < end and start <= c[2] < end:
            return chunk
        chunk += 1
    return None

def shift_overrides(data_dir, shift_dir, collection, findtext):
    for song in os.listdir(os.path.join(data_dir, collection)):
        if song != working_song and working_song != "": continue
        if ".DS_Store" in song: continue
        print('\t\t', song)
        shiftsong = song
        if collection == "Scherbaum Mshavanadze":
            shiftsong = song[:6]
        with open(os.path.join(shift_dir, collection, shiftsong, "shifts.txt")) as shift_file:
            for voice_type in os.listdir(os.path.join(data_dir, collection, song)):
                if "shifts" in voice_type or "shifted" in voice_type or "~" in voice_type or "#" in voice_type: continue
                if parse_name(song, voice_type) is None: continue
                    
                shiftList = list(map(float, shift_file.readline().split(' ')))
                it = iter(shiftList)
                shifts = list(zip(it, it, it))
                
                with open(os.path.join(data_dir, collection, song, voice_type)) as original:
                    with open(os.path.join(data_dir, collection, song, voice_type[:voice_type.index(findtext)] + "_shifted.txt"), "w") as new_shifted:
                        for line in original:
                            c = list(map(float, line.strip().split(' ')))
                            idx = chunkIndex(c, shifts)
                            if idx == None:
                                raise ValueError(f"Override {c} not in chunks {s}")
                            c[0] += shifts[idx][2]
                            c[2] += shifts[idx][2]
                            new_shifted.write(" ".join(list(map(str, c))) + '\n')


### Scherbaum Mshavanadze, Teach Yourself Gurian Songs, Teach Yourself Megrelian Songs

In [ ]:
collections = ['Scherbaum Mshavanadze', 'Teach Yourself Megrelian Songs', 'Teach Yourself Gurian Songs']
algos = ['boersma', 'crepe', 'hermes', 'noll', 'yin']
working_song = "Adila-Alipasha"
for collection in collections:
    print(f"Ground estimates for {collection}")
    shift_songs(data_dir, data_dir, collection, '.')
    
    print(f"Algorithms for {collection}")
    for algo in os.listdir(os.path.join(data2_dir)):
        if not algo in algos: continue
        print("\t", algo)
        shift_songs(data2_dir + '/' + algo, data_dir, collection, '.')

    print(f"Targets for {collection}")
    shift_songs(data3_dir, data_dir, collection, '.txt', match='M.txt')
    print(f"Notes for {collection}")
    shift_songs(data3_dir, data_dir, collection, '.txt', match='notes.txt')

    print(f"Overrides for {collection}")
    shift_overrides(data4_dir, data_dir, collection, '.')
    print('')


## Sandbox

In [ ]:
ndata_dir = "/Akamai/voice/data/ground-estimate/Teach Yourself Gurian Songs/Adila-Alipasha/AHDS2M.txt"
cdata = np.loadtxt(ndata_dir)
x1, y1 = cdata[:, 0], cdata[:, 1]
x2, y2 = align(x1, y1, 65, gurian=True)
np.savetxt("/Akamai/voice/data/ground-estimate/Teach Yourself Gurian Songs/Adila-Alipasha/AHDS2M_shifted.txt", np.vstack((x2, y2)).T, fmt='%.2f')

In [ ]:
data_dir = '/Akamai/voice/data/pitches-vuv-crepe-assisted/'
for algo in os.listdir(data_dir):
    for e in os.listdir(os.path.join(data_dir, algo, "Teach Yourself Megrelian Songs")):
        if e == "Scherbaum Mshavanadze":
            os.rmdir(os.path.join(data_dir, algo, "Teach Yourself Megrelian Songs", e))

In [ ]:
y = np.array((0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0))
alignChunk(y, -4)

In [ ]:
c = np.empty(0)
d = np.array((1,2,3,4))
np.append(c, d)